# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [35]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [36]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [63]:
berlin52 = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");

In [37]:
df_aleatorio = pd.read_csv("instancias_aleatorias/indice.csv")

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

- **VMC** : vecino mas cercano
- **AGM** : arbol generador minimo
- **I** : incersion
- **tabu falta** : tabu

In [38]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../tp2", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo = process.stdout.read() 
    #print(process.stdout.read()) # resultado del metodo
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    costo = costo.split(" ")[1]
    costo = costo.split("\n")[0]
    res = (tiempo_de_ejecucion,costo)
    
    return res;




## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [39]:
experimentos = [];
experimentos

[]

## Experimento aleatorio


In [40]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"VMC", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"]]);    

In [ ]:
## Tabu search

In [ ]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM-TS1", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"AGM-TS2", fila["archivo"]]);
    
    experimentos.append([fila["dataset"], fila["n"],"VCM-TS1", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"VCM-TS2", fila["archivo"]]);

    
    #     experimentos.append([fila["dataset"], fila["n"],"I", fila["archivo"]]); 

In [41]:
columnas = ["dataset", "n", "metodo", "tiempo","costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costo = 0
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[2], experimento[3])[0]
        costo = correr_experimento(experimento[2], experimento[3])[1]
        tiempos.append(t);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo,costo]);

'Experimento: 114/114'

In [42]:
df_resultados = pd.DataFrame(filas, columns=columnas);

df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [44]:
df_resultados

,dataset,n,metodo,tiempo,costo
0,dataset_aleatorias,10,AGM,0.018330,1231
1,dataset_aleatorias,10,VMC,0.008803,874
2,dataset_aleatorias,10,I,0.096841,77
3,dataset_aleatorias,15,AGM,0.023357,1396
4,dataset_aleatorias,15,VMC,0.012061,1235
...,...,...,...,...,...
109,dataset_aleatorias,190,VMC,1.219740,3259
110,dataset_aleatorias,190,I,1990.380000,-41569
111,dataset_aleatorias,195,AGM,1.414610,11912
112,dataset_aleatorias,195,VMC,1.474870,3560
